## Setup

The structural data used in this notebook was published as part of [Phys. Rev. B **95**, 094203](https://doi.org/10.1103/PhysRevB.95.094203)  
and is available from https://www.repository.cam.ac.uk/handle/1810/262814

In [1]:
from ase.io import read
import numpy as np

dataset = read("structures/labelled-gap17-test-set.extxyz", index=":")

dft_energy = np.array([structure.info["energy"] for structure in dataset])
gap_energy = np.array([structure.info["gap17_energy"] for structure in dataset])
atoms_per_cell = np.array([len(structure) for structure in dataset])

dft_forces = np.vstack([structure.arrays["force"] for structure in dataset])
gap_forces = np.vstack([structure.arrays["gap17_force"] for structure in dataset])

# Error Metrics for Scalar Properties

The *error* when predicting a scalar metric is easy to quantify: it's the difference between the predicted value and the true value. This is termed the residual, $r_i = y_i - \hat{y}_i$. Applying summary metrics directly to these residuals is straightforward: 
$$\text{MAE} = \frac{1}{N}\sum_{i=1}^N |r_i| \cdot w_i
\quad \text{and} \quad 
\text{RMSE} = \sqrt{\frac{1}{N}\sum_{i=1}^N r_i^2 \cdot w_i^2}$$
where $w_i$ is the weight of the $i$'th data point.

In [2]:
# some basic implementations of error metrics:

def mae(residuals):
    return np.mean(np.abs(residuals))

def rmse(residuals):
    return np.sqrt(np.mean(residuals**2))

### Per-cell Energy Errors:

In [3]:
per_cell_energy_residuals = dft_energy - gap_energy

_mae = mae(per_cell_energy_residuals)
_rmse = rmse(per_cell_energy_residuals)

print("MAE: ", round(_mae, 3), "eV/cell")
print("RMSE:", round(_rmse, 3), "eV/cell")

MAE:  2.261 eV/cell
RMSE: 3.217 eV/cell


### Scaled Energy Errors:
To obtain an energy error that is independent of the number of atoms in a cell, we divide by the square root of the number of atoms in the cell:

In [4]:
weights = 1 / np.sqrt(atoms_per_cell)

_mae = mae(per_cell_energy_residuals * weights)
_rmse = rmse(per_cell_energy_residuals * weights)

print("MAE: ", round(_mae, 3), "eV/atom")
print("RMSE:", round(_rmse, 3), "eV/atom")

MAE:  0.272 eV/atom
RMSE: 0.377 eV/atom


# Error Metrics for Vector Properties

We can extend the idea of scalar residuals into $N>1$ dimensions by defining a residual vector as $\mathbf{r}_i = \mathbf{y}_i - \hat{\mathbf{y}}_i$. Due to the vector nature of the resulting residuals, we are now presented with several ways to apply the "usual" summary metrics to collections of residuals.

In [5]:
force_residuals = dft_forces - gap_forces

# there are 28,337 atomic environments in the test set
# and therefore we have 28,337 force residuals, each of which
# is a 3-vector
force_residuals.shape

(28337, 3)

## Error Magnitudes

The simplest approach is to simply apply the scalar metrics to the magnitude (L2 norm) of the residual vector, $r_i = ||\mathbf{r}_i||_2$. The resulting metrics are:

$$
\text{MAE}_{\text{magnitudes}}(\mathbf{r}) = \frac{1}{N}\sum_{i=1}^N ||\mathbf{r}_i||_2 
$$

and

$$
\text{RMSE}_{\text{magnitudes}}(\mathbf{r}) = \sqrt{\frac{1}{N}\sum_{i=1}^N {||\mathbf{r}_i||_2}^2}  = \sqrt{\frac{1}{N}\sum_{i=1}^N r_{i,x}^2 + r_{i,y}^2 + r_{i,z}^2}
$$

In [6]:
# we can take the (L2) norm of each force residual to get
# the euclidean magnitude of the force residual
# we can interpret this as the distance between the DFT and GAP forces

force_residuals_magnitudes = np.linalg.norm(force_residuals, axis=1)
force_residuals_magnitudes.shape

(28337,)

In [7]:
# now that we have scalar values, we can use the normal MAE and RMSE metrics
mag_mae = mae(force_residuals_magnitudes)
mag_rmse = rmse(force_residuals_magnitudes)

print("MAE: ", round(mag_mae, 3), "eV/Å")
print("RMSE:", round(mag_rmse, 3), "eV/Å")

MAE:  1.667 eV/Å
RMSE: 1.892 eV/Å


## Error Vector Components

An alternative approach is to apply the scalar metrics to each component of the residual vectors:

$$
\begin{align*}
\text{MAE}_\text{component-wise} &= \text{MAE}([\mathbf{r}_x; \mathbf{r}_y; \mathbf{r}_z])\\
&= \frac{1}{3N}\sum_{i=1}^N |r_{i,x}| + |r_{i,y}| + |r_{i,z}|\\
&= \frac{1}{N}\sum_{i=1}^N \frac{1}{3}||\mathbf{r}_i||_1\\
\end{align*}  
$$  
and
$$
\begin{align*}  
\text{RMSE}_\text{component-wise} &= \text{RMSE}([\mathbf{r}_x ; \mathbf{r}_y; \mathbf{r}_z])\\
&= \sqrt{\frac{1}{3N}\sum_{i=1}^N r_{x,i}^2 + r_{y,i}^2 + r_{z,i}^2}\\
&= \frac{1}{\sqrt{3}}\sqrt{\frac{1}{N}\sum_{i=1}^N {||\mathbf{r}_i||_2}^2} \equiv \frac{1}{\sqrt{3}} \text{RMSE}_{\text{magnitude}}\\
\end{align*}
$$

where we use $[a;b;...]$ to denote the concatenation of vectors.

A geometric interpretation of the component-wise MAE is therefore one third of the average Manhattan distance (L1 norm) between the predicted and true vectors. 

The component-wise RMSE is the average Euclidean distance (L2 norm) between the predicted and true vectors.

In [8]:
force_residuals_components = force_residuals.reshape(-1)
force_residuals_components.shape

(85011,)

In [9]:
comp_mae = mae(force_residuals_components)
comp_rmse = rmse(force_residuals_components)

print("MAE: ", round(comp_mae, 3), "eV/Å")
print("RMSE:", round(comp_rmse, 3), "eV/Å")

MAE:  0.833 eV/Å
RMSE: 1.092 eV/Å


In [10]:
# the magnitude and component RMSEs are related by a factor of √3 
# due to Pythagoras' theorem
np.isclose(mag_rmse / (3**0.5), comp_rmse)

True

## On Rotational Invariance

An important thing to note is that, due to pythagoras' theorem, the L2 norm of a vector is invariant to rotations: 

$$
||\mathbf{v}||_2 \equiv ||\mathcal{R}(\mathbf{v})||_2 \quad \forall \quad \mathbf{v} \in \mathbb{R}^3, \mathcal{R} \in \text{SO}(3)
$$

and, hence, all metrics applied to the L2 norm are also rotationally invariant.

This is not the case for the Manhattan distance (L1) norm however! For an N-D vector of euclidean length (L2) $1$, the L1 norm is bounded by $1$ (the vector points along a single axis). However, for the same vector, the L1 norm is bounded by $\sqrt{N}$ (the vector points along the diagonal of an $N$-dimensional hypercube). This means that the component-wise MAE is not rotationally invariant.

We show this result for the 2d case in `demo-rotation-invariance.ipynb`.

In [12]:
def randomly_rotate(thing):
    """randomly rotate a 3D vector or matrix"""

    from scipy.stats import special_ortho_group
    
    rotation_matrix = special_ortho_group.rvs(3)
    return np.dot(rotation_matrix, thing.T).T


rotated_comp_maes = []

for _ in range(100):
    rotated_force_residuals = randomly_rotate(force_residuals)
    rotated_comp_mae = mae(rotated_force_residuals.reshape(-1))
    rotated_comp_maes.append(rotated_comp_mae)

np.min(rotated_comp_maes), np.max(rotated_comp_maes)

(0.8324940950668309, 0.834605631452439)

In [16]:
percentage_variation = (
    (np.max(rotated_comp_maes) - np.min(rotated_comp_maes)) 
    / np.mean(rotated_comp_maes)
)

# this is a relatively small variation relative to the size of the error
# but could be orders of magnitude larger given a different dataset
f"{percentage_variation:.2%}"

'0.25%'

In this dataset, we deal with structures with no inherent orientation relative to the cartesian axes. Coupled with the large number of atomic environments that we average over, this leads to a small variation in the MAEs as a function of 3d rotation. We stress, however, that this will not always be the case. For instance, if the dataset is for crystalline structures with the majority of the bonds, and therefore the forces, oriented with the Cartesian axes. In this case, where errors may lie predominantly along lattice vectors, the component-wise MAE will have a near maximal variation ($\sqrt{3}$) as a function of rotation!


## Final Remarks

We note a common pitfall: under the hood, without explicitly incorporating the L2 norms of errors, `numpy` performs componentwise MAE/RMSE operations:

In [13]:
force_residuals_components.shape, mae(force_residuals_components)

((85011,), 0.8329790830584277)

In [14]:
# note the MAE is identical above and below, despite the different shapes
# i.e. even without explicit reshaping, the MAE is computed over the flattened arrays
force_residuals.shape, mae(force_residuals)

((28337, 3), 0.8329790830584277)